In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head(20)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
5,25000.0,0.1797,634.43,MORTGAGE,90000.0,Source Verified,Mar-2019,low_risk,n,18.49,...,93.1,66.7,0.0,0.0,550717.0,418934.0,18800.0,369051.0,N,N
6,20400.0,0.2000,540.48,RENT,51315.0,Source Verified,Mar-2019,low_risk,n,25.63,...,100.0,0.0,1.0,0.0,40023.0,23469.0,3500.0,27523.0,N,N
7,10000.0,0.1557,349.46,OWN,66000.0,Source Verified,Mar-2019,low_risk,n,9.00,...,100.0,16.7,0.0,0.0,29300.0,22377.0,8400.0,12500.0,N,N
8,14000.0,0.0881,443.96,RENT,45000.0,Not Verified,Mar-2019,low_risk,n,10.24,...,100.0,0.0,0.0,0.0,25100.0,9724.0,12100.0,3000.0,N,N
9,10000.0,0.1474,345.39,RENT,63295.0,Verified,Mar-2019,low_risk,n,39.34,...,97.2,71.4,0.0,0.0,119822.0,102924.0,15300.0,80722.0,N,N


# Split the Data into Training and Testing

In [6]:
status_c = {
    "low_risk": 0,
    "high_risk": 1
}

In [7]:
# Create our features
X = pd.get_dummies(df.drop('loan_status',axis=1))

# Create our target
y = df[['loan_status']]
y['loan_status'] = df["loan_status"].apply(lambda x: status_c[x])
y.head()

,loan_status
0,0
1,0
2,0
3,0
4,0


In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [9]:
# Check the balance of our target values
y['loan_status'].value_counts()

0    68470
1      347
Name: loan_status, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [11]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=50, random_state=1)
model

RandomForestClassifier(n_estimators=50, random_state=1)

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
#scaling data
data_scaler = StandardScaler()
X_scaler = data_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train) 
X_test_scaled = X_scaler.transform(X_test)

In [14]:
model = model.fit(X_train_scaled,y_train)

In [15]:
pred = model.predict(X_test_scaled)

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report 
acc_score = accuracy_score(y_test, pred)
print(f'accuracy score is {acc_score}')

accuracy score is 0.9959895379250218


In [17]:
# Display the confusion matrix
cm = confusion_matrix(y_test, pred)
cm_df = pd.DataFrame(cm, index=['Acutal 0',"Actual 1"], columns=['Predicuted 0','Predicted 1'])
cm_df

,Predicuted 0,Predicted 1
Acutal 0,17101,3
Actual 1,66,35


In [18]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, pred))

Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17104
           1       0.92      0.35      0.50       101

    accuracy                           1.00     17205
   macro avg       0.96      0.67      0.75     17205
weighted avg       1.00      1.00      1.00     17205



In [19]:
importances = model.feature_importances_
importances

array([0.01499202, 0.01558514, 0.01852927, 0.01298759, 0.01736417,
       0.00341155, 0.00461508, 0.00905752, 0.0020595 , 0.01390742,
       0.01191967, 0.02205958, 0.01907131, 0.06674817, 0.06153379,
       0.06158835, 0.05960669, 0.01279472, 0.        , 0.        ,
       0.07072931, 0.00159359, 0.        , 0.        , 0.00551732,
       0.01474994, 0.00560593, 0.00679746, 0.00369526, 0.00693811,
       0.01090715, 0.01354436, 0.0118057 , 0.00564144, 0.00803271,
       0.01797751, 0.01154567, 0.01437532, 0.00763071, 0.00903229,
       0.0088064 , 0.00885882, 0.01479031, 0.01440632, 0.01447846,
       0.0003381 , 0.        , 0.01255462, 0.01542977, 0.00813502,
       0.00967922, 0.0080369 , 0.00923557, 0.01074511, 0.00355593,
       0.00611748, 0.00686624, 0.00781314, 0.00789963, 0.01134608,
       0.00733099, 0.01014139, 0.00773768, 0.00891217, 0.        ,
       0.        , 0.00111766, 0.00644775, 0.00993464, 0.00498071,
       0.0036057 , 0.        , 0.01510209, 0.01424265, 0.01330

In [20]:
# List the features sorted in descending order by feature importance
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.07072931115642557, 'last_pymnt_amnt'),
 (0.06674816539749895, 'total_pymnt'),
 (0.061588350176476035, 'total_rec_prncp'),
 (0.06153379103133279, 'total_pymnt_inv'),
 (0.05960669296964875, 'total_rec_int'),
 (0.022059577388587305, 'out_prncp'),
 (0.019071312207782406, 'out_prncp_inv'),
 (0.018529267151321548, 'installment'),
 (0.017977514227565397, 'max_bal_bc'),
 (0.017364171803943312, 'dti'),
 (0.015585135571353593, 'int_rate'),
 (0.015429765422234641, 'mo_sin_old_rev_tl_op'),
 (0.015102090349959154, 'tot_hi_cred_lim'),
 (0.014992021681456559, 'loan_amnt'),
 (0.01479030864836467, 'avg_cur_bal'),
 (0.014749935102412745, 'tot_cur_bal'),
 (0.01447846224240306, 'bc_util'),
 (0.014406318931342676, 'bc_open_to_buy'),
 (0.01437531553321221, 'total_rev_hi_lim'),
 (0.014242645993736565, 'total_bal_ex_mort'),
 (0.01390742330265004, 'revol_bal'),
 (0.01384606966557047, 'total_il_high_credit_limit'),
 (0.013544362651394201, 'total_bal_il'),
 (0.013306691512175122, 'total_bc_limit'),
 (0.01298

### Easy Ensemble AdaBoost Classifier

In [22]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
# from sklearn.ensemble
model = EasyEnsembleClassifier()
model = model.fit(X_train_scaled,y_train)

In [23]:
pred=model.predict(X_test_scaled)

In [24]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, pred)
print(f'accuracy score is {acc_score}')

accuracy score is 0.9461784365010172


In [25]:
# Display the confusion matrix
cm = confusion_matrix(y_test, pred)
cm_df = pd.DataFrame(cm, index=['Acutal 0',"Actual 1"], columns=['Predicuted 0','Predicted 1'])
cm_df

,Predicuted 0,Predicted 1
Acutal 0,16188,916
Actual 1,10,91


In [26]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, pred))

Classification Report
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     17104
           1       0.09      0.90      0.16       101

    accuracy                           0.95     17205
   macro avg       0.54      0.92      0.57     17205
weighted avg       0.99      0.95      0.97     17205

